# 整体架构和例子

## Future 机制
```mermaid
graph TD;
    subgraph Future 机制
        direction TB
        subgraph future.hpp
            direction TB; %% 设置A1内部子模块的布局方向为从上到下
            class_future["<b>类 future</b><br> future_impl&ltfuture_result_type&gt impl"];
            function_schedule["<b>方法 schedule"];
        end
        
        subgraph detail/future.hpp
            direction TB; %% 设置A1内部子模块的布局方向为从上到下
            class_future_impl["<b>类 future_impl</b><br>bool m_ready<br>future_result_type m_result<br>mutex m_monitor<br>m_condition_ready<br>bool m_is_cancelled<br>bool m_executing"];
            function_future_impl_task_func["<b>方法 future_impl_task_func"];
        end
    end

    class_future --> class_future_impl

    %% 为每个模块添加样式（可选）
    style future.hpp fill:#f9f,stroke:#333,stroke-width:2px;
    style detail/future.hpp fill:#bbf,stroke:#333,stroke-width:2px;
```


## 线程池

```mermaid
graph TD;
    subgraph thread_pool["pool.hpp：类 thread_pool"]
        direction TB;
        m_core["shared_ptr<pool_core_type> m_core"];
        m_shutdown_controller["shared_ptr<void> m_shutdown_controller"];
    end

    subgraph pool_core["detail/pool_core.hpp:&nbsp;类&nbsp;pool_core"]
        direction TB;
        m_worker_count["m_worker_count：当前线程数"];
	    m_target_worker_count["m_target_worker_count：目标线程数"];
	    m_active_worker_count["m_active_worker_count：当前不在休息的线程数"];  
        %% 任务调度器实例                         
        m_scheduler["m_scheduler"] 
	    m_size_policy["m_size_policy：指向大小策略的智能指针"];
	    m_terminate_all_workers["m_terminate_all_workers：是否触发了所有工作线程的终止"];
        %% 已终止但未完全销毁的工作线程列表
        m_terminated_workers["std::vector<shared_ptr<worker_type> > m_terminated_workers"]; 
	    m_monitor["m_monitor：保护m_scheduler的递归互斥锁"];
	    m_worker_idle_or_terminated_event["m_worker_idle_or_terminated_event：有线程空闲或终止的条件变量"];
        m_task_or_terminate_workers_event["m_task_or_terminate_workers_event：任务队列添加了新任务或终止所有线程的条件变量"];
        
        m_worker_count ~~~ m_target_worker_count;
        m_target_worker_count ~~~ m_active_worker_count;
        m_active_worker_count ~~~ m_scheduler;
        m_scheduler ~~~ m_size_policy;
        m_size_policy ~~~ m_terminate_all_workers;
        m_terminate_all_workers ~~~ m_terminated_workers;
        m_terminated_workers ~~~ m_monitor;
        m_monitor ~~~ m_worker_idle_or_terminated_event;
        m_worker_idle_or_terminated_event ~~~ m_task_or_terminate_workers_event;
    end

    subgraph scheduler["scheduling_policies.hpp"]
        direction TB;
        fifo_scheduler["类 fifo_scheduler"];
        lifo_scheduler["类 lifo_scheduler"];
        prio_scheduler["类 prio_scheduler"];

        fifo_scheduler ~~~ lifo_scheduler;
        lifo_scheduler ~~~ prio_scheduler;
    end

    static_size["size_policies.hpp：类 static_size"]

    subgraph worker_thread["worker_thread.hpp:类worker_thread"]
        direction TB;
        scheduler_m_pool["m_pool：指向pool_type的共享指针"]
        m_thread["m_thread：指向实际的执行线程boost::thread"]

        scheduler_m_pool ~~~ m_thread;
    end

    m_core --> pool_core

    m_scheduler --> scheduler

    m_size_policy --> static_size

    m_terminated_workers --> worker_thread
    scheduler_m_pool --> pool_core
```

## 例子
```cpp
#include <iostream>
#include <string>
#include <vector>
#include "threadpool/pool.hpp"
#include "threadpool/future.hpp"
#include <boost/thread.hpp>
#include <boost/bind.hpp>

// 1. 无返回值的任务：模拟写入日志
void log_event(int event_id, const std::string& message) {
    // 模拟I/O操作的耗时
    boost::this_thread::sleep(boost::posix_time::milliseconds(500)); 
    
    // 使用互斥锁保护std::cout，防止多线程输出混乱
    static boost::mutex cout_mutex;
    boost::lock_guard<boost_mutex> lock(cout_mutex);
    std::cout << "[Log Task " << event_id << "] Thread ID: " 
              << boost::this_thread::get_id() << " -> " << message << std::endl;
}

// 2. 有返回值的任务：模拟数据计算
long long fibonacci(int n) {
    // 模拟计算密集型操作的耗时
    boost::this_thread::sleep(boost::posix_time::seconds(1)); 
    
    if (n <= 1) return n;
    long long a = 0, b = 1;
    for (int i = 2; i <= n; ++i) {
        long long temp = a + b;
        a = b;
        b = temp;
    }
    
    static boost::mutex cout_mutex;
    boost::lock_guard<boost_mutex> lock(cout_mutex);
    std::cout << "[Calc Task] Thread ID: " << boost::this_thread::get_id() 
              << " -> Calculated Fibonacci(" << n << ")" << std::endl;

    return b;
}


int main() {
    // 1. 创建一个拥有4个工作线程的线程池
    boost::threadpool::pool tp(4);
    // 2. 提交多个无返回值的任务
    for (int i = 1; i <= 5; ++i) {
        tp.schedule(boost::bind(log_event, i, "User logged in."));
    }
    // 3. 提交多个有返回值的任务，并保存它们的 future
    std::vector<boost::threadpool::future<long long>> futures;
    // 使用全局的 schedule 函数来获取 future 对象
    futures.push_back(boost::threadpool::schedule(tp, boost::bind(fibonacci, 30)));
    futures.push_back(boost::threadpool::schedule(tp, boost::bind(fibonacci, 35)));
    futures.push_back(boost::threadpool::schedule(tp, boost::bind(fibonacci, 40)));
    // 4. 等待并获取所有有返回值任务的结果
    for (size_t i = 0; i < futures.size(); ++i) {
        // fut.get() 会阻塞，直到这个特定的任务完成
        long long result = futures[i].get(); 
        std::cout << "主线程：成功获取到第 " << i+1 << " 个计算结果 -> " << result << std::endl;
    }

    // 5. 等待所有任务（包括无返回值的）都执行完毕
    tp.wait(); // 确保所有任务都结束了
    std::cout << "主线程：所有任务均已完成，程序即将退出。" << std::endl;
    // 当 main 函数结束，tp 对象离开作用域时，线程池会自动安全关闭
    return 0;
}
```

# 语法相关

## C++ 中四种类型转换操作符
统一语法：`static/dynamic/const/reinterpret_cast<new_type>(expression)`

`static_cast`：处理编译时就能确定其合法性的转换
- 数值类型转换
- 类型和对应类型指针、指针和void*
    ```cpp
    int x = 10;
    void* vp = static_cast<void*>(&x);
    int* ip = static_cast<int*>(vp);
    ```
- 上行转换：将子类的指针/引用转换为父类的指针/引用

`dynamic_cast`：处理多态中的下行转换
- 只能用于指针/引用，并且基类中必须至少有一个虚函数

`const_cast`：用于移除 const 或 volatile
- 例如
    ```cpp
    const char* my_string = "Hello, World!";
    char* str = const_cast<char*>(my_string)
    ```
`reinterpret_cast`：最强的转换，底层的、纯粹的按位重新解释
- 例如
    ```cpp
    const char* my_string = "Hello, World!";
    char* str = const_cast<char*>(my_string)
    ```

# 工具（detail/locking_ptr.hpp、scope_guard.hpp）

## detail/locking_ptr.hpp
核心是 *带作用域锁定机制的智能指针 `locking_ptr`*
- 包含成员 T* m_obj 和 Mutex & m_mutex
- 当一个 `locking_ptr` 对象创建时 m_mutex 会自动加锁，而销毁时（离开作用域）会自动解锁
- 使用场景
  - 当有一个共享数据 shared_data 需要锁 lock 来保证安全访问时
  - 不需要多次手动对 lock 加锁解锁，而只需要在访问该数据的地方定义 locking_ptr 对象即可。例如：
    ```cpp
    volatile int shared_data = 100;
    boost::mutex data_mutex;

    // 函数 B1：一个线程会执行的操作
    void worker_function_B1() {
        boost::threadpool::detail::locking_ptr<int, boost::mutex> p(shared_data, data_mutex);
        *p = *p + 10;
        // 当 p 离开作用域时，自动解锁
    } // B1 的锁在这里自动释放

    // 函数 B2：另一个线程会执行的操作
    void worker_function_B2() {
        // B2 会在这里阻塞，直到 B1 释放锁
        boost::threadpool::detail::locking_ptr<int, boost::mutex> p(shared_data, data_mutex);
        *p = *p * 2;
        // 当 p 离开作用域时，自动解锁
    } // B2 的锁在这里自动释放

    int main() {
        // 创建并启动两个线程
        boost::thread thread1(worker_function_B1);
        boost::thread thread2(worker_function_B2);

        // 等待两个线程完成
        thread1.join();
        thread2.join();

        return 0;
    }
    ```

### 语法相关

#### `class locking_ptr: private noncopyable`
首先注意：*基类的 `private` 成员对于子类来说是不可见的*。

`locking_ptr` 类没有声明自己的拷贝构造函数或赋值运算符。
当编译器尝试为 `locking_ptr` 自动生成一个拷贝构造函数时，它会发现需要调用基类 `noncopyable` 的拷贝构造函数来完成基类的部分初始化

由于 `noncopyable` 的拷贝构造函数是 `private` 的，`locking_ptr` 无法访问它即编译器无法完成这个自动生成的任务，从而达到了禁止复制的目的。

### 带作用域锁定机制的智能指针 `locking_ptr`
```cpp
/* 带作用域锁定机制的智能指针
 * 通过在构造时锁定互斥量、析构时解锁互斥量
 * 来实现对内部指针的同步访问。这确保了在智能指针存在期间，对象访问是线程安全的。
 * \tparam T 被指向对象的类型
 * \tparam Mutex 互斥锁的类型
 */
template <typename T, typename Mutex>
class locking_ptr: private noncopyable
{
    T* m_obj;                     // 指向实例的指针
    Mutex & m_mutex;              // 用于作用域锁定的互斥量引用

    public:
    /* 构造函数
     * 创建锁定指针并立即锁定关联的互斥量。在构造函数完成后，对象的访问将是线程安全的。
     * \param obj 要访问的volatile对象引用
     * \param mtx 用于同步的volatile互斥量引用
     */
    locking_ptr(volatile T& obj, const volatile Mutex& mtx)
        : m_obj(const_cast<T*>(&obj))
        , m_mutex(*const_cast<Mutex*>(&mtx))
    {   
        // 锁定互斥量，确保线程安全访问
        m_mutex.lock();
    }

    /* 析构函数
     * 销毁锁定指针并自动解锁关联的互斥量。这确保了RAII模式的正确实现，即使在异常情况下也能保证互斥量被正确释放。
     */
    ~locking_ptr()
    { 
        // 解锁互斥量，释放同步资源
        m_mutex.unlock();
    }

    /* 解引用操作符
     * 返回对存储实例的引用。由于互斥量已被锁定，对返回引用的访问是线程安全的。
     * \return 存储实例的引用
     */
    T& operator*() const
    {    
        return *m_obj;    
    }

    /* 成员访问操作符
     * 回指向存储实例的指针。由于互斥量已被锁定，通过返回指针对成员的访问是线程安全的。
     * \return 指向存储实例的指针
     */
    T* operator->() const
    {   
        return m_obj;   
    }
};
```

## detail/scope_guard.hpp
核心是 *作用域守卫类 `scope_guard`*
- 包含成员 `const function0<void> m_function` 和 `bool m_is_active`
- 当一个 `scope_guard` 对象销毁时：如果 `m_is_active` 为真则 `m_function` 会自动调用
- 作用是：确保在离开一个代码作用域时，无论如何都会自动执行某个预定的操作

### 作用域守卫类 `scope_guard`
```cpp
/* 作用域守卫类
 * 在对象析构时自动执行指定的清理函数。
 * 这确保了即使在异常情况下也能正确执行清理操作。该类不可拷贝，保证了清理操作的唯一性和安全性。
 */
class scope_guard: private boost::noncopyable
{
	const function0<void> m_function;  // 在作用域结束时要执行的清理函数
	bool m_is_active;                  // 标记守卫是否处于活动状态

public:
	/* 构造函数，创建一个作用域守卫对象，指定在作用域结束时要执行的清理函数。
	 * \param call_on_exit 在作用域结束时要调用的函数对象
	 */
	scope_guard(function0<void> const & call_on_exit)
	: m_function(call_on_exit)
	, m_is_active(true)
	{
	}

	/* 析构函数，在对象销毁时自动调用。
     * 如果守卫处于活动状态且清理函数有效，则执行清理函数。这确保了资源的正确释放。
	 */
	~scope_guard()
	{
		if(m_is_active && m_function)
		{
			m_function();
		}
	}

	/* 禁用守卫
	 * 手动禁用作用域守卫，使其在析构时不执行清理函数。
	 * 这在某些情况下很有用，比如操作成功完成时不需要执行清理。
	 */
	void disable()
	{
		m_is_active = false;
	}
};
```

# Future 机制（future.hpp和detail/future.hpp）

## future.hpp

### 语法相关

#### disable_if
`Function` 返回类型不为 void 时，使用 `future<Function 的返回类型>` 作为 `schedule` 的返回类型
```cpp
template<class Pool, class Function>
typename disable_if < 
  is_void< typename result_of< Function() >::type >,
  future< typename result_of< Function() >::type >
>::type
schedule(Pool& pool, const Function& task) {...}
```
其中
- `Function` 是个可调用类型
- `result_of< Function() >::type` 是获取 `Function` 被调用时的返回类型
- `is_void< typename T >`：如果类型 `T` 是 void 返回 true，否则返回 false
- `future< typename result_of< Function() >::type >`：模板类 `future` 的模板参数被确定为 `Function` 被调用时的返回类型而构成的类
- `disable_if<bool condition, typename T>`
  - 如果 `condition` 为 false，使用 T
  - 否则失败，该 `schedule` 函数被编译器忽略，编译器会寻找重载版本
    - 注意：**C++ 中函数重载是：函数名相同，参数列表不同（包括参数类型和参数名）**

### 将任务函数包装为 Future 任务函数并添加到线程池的任务队列 `schedule`
流程：
- 基于 `Function` 的返回类型 `future_result_type` 构建 ***Future 实现对象*** `impl: future_impl<future_result_type>`
- 基于 Future 实现对象 `impl` 构建 ***Future 对象*** `res: future <future_result_type>`
- 基于任务函数 `task: Function` 和 Future 实现对象 `impl` 构建 ***Future 任务函数***，并使用 `pool.schedule` 传递到线程池的任务队列中
- 返回 Future 对象 `res`
```cpp
/* 将任务函数包装为 Future 任务函数并添加到线程池的任务队列
 * \tparam Pool 线程池类型
 * \tparam Function 任务函数类型
 * \param pool 线程池引用
 * \param task 要执行的任务函数
 * \return 与任务关联的Future对象，用于获取执行结果
 */
template<class Pool, class Function>
typename disable_if < 
  is_void< typename result_of< Function() >::type >,
  future< typename result_of< Function() >::type >
>::type
schedule(Pool& pool, const Function& task)
{
  // 定义Future结果类型
  typedef typename result_of< Function() >::type future_result_type;

  // 创建Future实现对象和Future对象
  shared_ptr<detail::future_impl<future_result_type> > impl(new detail::future_impl<future_result_type>);
  future <future_result_type> res(impl);

  // 将任务包装成Future任务函数并提交到线程池
  pool.schedule(detail::future_impl_task_func<detail::future_impl, Function>(task, impl));

  // 返回Future对象供调用者使用
  return res;
}

```

### Future 对象（异步任务执行结果容器）`future`
内部包含一个智能指针，指向一个 **Future实现对象** `m_impl: future_impl<Result>`，参考 [detail/future.hpp](./detail/future.hpp)。
```cpp
/* 异步任务结果容器类
 * \tparam Result 任务返回值的类型 
 */ 
template<class Result> 
class future
{
private:
  /// Future实现的智能指针，使用Pimpl模式隐藏实现细节
  shared_ptr<detail::future_impl<Result> > m_impl;

public:
    /// 结果类型定义，返回常量引用以避免拷贝
    typedef Result const & result_type;
    /// Future结果类型定义
    typedef Result future_result_type;

public:
  /* 默认构造函数，内部创建一个新的Future对象，内部创建对应的实现对象 */
  future()
  : m_impl(new detail::future_impl<future_result_type>())
  {
  }

  /* 构造函数，使用已存在的实现对象创建Future，主要用于内部实现。
   * \param impl Future实现对象的智能指针
   */
  future(shared_ptr<detail::future_impl<Result> > const & impl)
  : m_impl(impl)
  {
  }

  /* 检查任务是否已完成
   * \return true表示任务已完成，false表示任务仍在执行中
   */
  bool ready() const
  {
    return m_impl->ready();
  }

  /* 等待任务完成，阻塞当前线程直到异步任务执行完成 */
  void wait() const
  {
    m_impl->wait();
  }

  /* 带超时的等待任务完成，阻塞当前线程直到异步任务执行完成或者超时。
   * \param timestamp 超时时间戳
   * \return true表示任务在超时前完成，false表示超时
   */
  bool timed_wait(boost::xtime const & timestamp) const
  {
    return m_impl->timed_wait(timestamp);
  }

  /* 获取任务执行结果（函数调用操作符）。如果任务尚未完成，将阻塞等待。
   * \return 任务执行的结果
   */
   result_type operator()()
   {
     return (*m_impl)();
   }

  /* 获取任务执行结果。如果任务尚未完成，将阻塞等待。
   * \return 任务执行的结果
   */
   result_type get()
   {
     return (*m_impl)();
   }

  /* 取消任务执行
   * 如果任务尚未开始执行或正在执行中，可以被取消。
   * \return true表示取消成功，false表示取消失败（任务可能已完成）
   */
   bool cancel()
   {
     return m_impl->cancel();
   }

  /* 检查任务是否已被取消 */
   bool is_cancelled() const
   {
     return m_impl->is_cancelled();
   }
};
``` 


## detail/future.hpp

### 异步任务结果容器实现类 `future_impl`

```cpp
/* 异步任务结果容器实现类（Future 实现对象）
 * 实现了Future模式的核心功能，用于存储和管理异步任务的结果。
 * 它提供了线程安全的结果设置、获取、等待和取消操作。
 * \tparam Result 异步任务的结果类型
 */
template<class Result> 
class future_impl
{
public:
  /// 结果类型定义（常量引用）
  typedef Result const & result_type;
  /// Future结果类型定义
  typedef Result future_result_type;
  /// Future类型定义
  typedef future_impl<future_result_type> future_type;

private:
    volatile bool m_ready;                  // 标示结果是否已准备就绪
    volatile future_result_type m_result;   // 存储异步任务的结果

    mutable mutex m_monitor;                // 保护共享状态的互斥锁
    mutable condition m_condition_ready;	  // 用于通知结果就绪的条件变量

    volatile bool m_is_cancelled;           // 标示任务是否已被取消
    volatile bool m_executing;              // 标示任务是否正在执行

public:
  /* 默认构造函数
   * 初始化Future实现对象，设置初始状态为未就绪且未取消。
   */
  future_impl()
  : m_ready(false)          // 初始化为未就绪状态
  , m_is_cancelled(false)   // 初始化为未取消状态
  {
  }

  /* 检查结果是否已就绪 */
  bool ready() const volatile
  {
    return m_ready; 
  }

  /* 等待结果就绪：阻塞当前线程直到异步任务完成并且结果可用 */
  void wait() const volatile
  {
    const future_type* self = const_cast<const future_type*>(this);
    mutex::scoped_lock lock(self->m_monitor);

    // 循环等待直到结果就绪
    while(!m_ready)
    {
      self->m_condition_ready.wait(lock);
    }
  }

  /* 带超时的等待结果就绪：阻塞当前线程直到异步任务完成或达到指定的超时时间。
   * \param timestamp 超时时间戳
   * \return 如果在超时前结果就绪则返回true，否则返回false
   */
  bool timed_wait(boost::xtime const & timestamp) const
  {
    const future_type* self = const_cast<const future_type*>(this);
    mutex::scoped_lock lock(self->m_monitor);

    // 循环等待直到结果就绪或超时
    while(!m_ready)
    {
      if(!self->m_condition_ready.timed_wait(lock, timestamp)) return false;
    }

    return true;
  }

  /* 获取异步任务的结果
   * 等待任务完成并返回结果。如果任务尚未完成，则阻塞当前线程。
   * \return 异步任务的结果引用
   */
  result_type operator()() const volatile
  {
    // 等待结果就绪
    wait();
    
    // 注释的异常处理代码，预留给将来的扩展
    /*
    if( throw_exception_ != 0 )
    {
      throw_exception_( this );
    }
    */
 
    // 返回结果的常量引用
    return *(const_cast<const future_result_type*>(&m_result));
  }

  /* 设置异步任务的结果
   * 由执行任务的线程调用，用于设置任务的执行结果。
   * 只有在任务未就绪且未被取消的情况下才会设置结果。、
   * \param r 要设置的结果值
   */
  void set_value(future_result_type const & r) volatile
  {
    locking_ptr<future_type, mutex> lockedThis(*this, m_monitor);
    // 只有在未就绪且未取消时才设置结果
    if(!m_ready && !m_is_cancelled)
    {
      lockedThis->m_result = r;                           // 设置结果值
      lockedThis->m_ready = true;                         // 标记为就绪
      lockedThis->m_condition_ready.notify_all();         // 通知所有等待的线程
    }
  }

  // 注释的异常处理方法，预留给将来的扩展
  /*
  template<class E> void set_exception() // throw()
  {
    m_impl->template set_exception<E>();
  }

  template<class E> void set_exception( char const * what ) // throw()
  {
    m_impl->template set_exception<E>( what );
  }
  */

   /* 尝试取消异步任务
    * 尝试取消尚未完成的异步任务。只有在任务未就绪或正在执行时才能取消。
    * \return 如果成功取消则返回true，否则返回false
    */
   bool cancel() volatile
   {
     // 只有在未就绪或正在执行时才能取消
     if(!m_ready || m_executing)
     {
        m_is_cancelled = true;
        return true;
     }
     else
     {
       return false;
     }
   }

   /* 检查任务是否已被取消 */
   bool is_cancelled() const volatile
   {
     return m_is_cancelled;
   }

   /* 设置任务的执行状态
    * 由任务执行线程调用，用于标记任务的执行状态。
    * \param executing 如果任务正在执行则为true，否则为false
    */
   void set_execution_status(bool executing) volatile
   {
     m_executing = executing;
   }
};
```

### Future 任务函数类 `future_impl_task_func`
包含两个成员：
- 要执行的函数对象 `function_type m_function`
- 关联的Future对象智能指针 `shared_ptr<future_type> m_future`
```cpp
/* Future任务函数
 * 将普通的函数对象适配为可以与 Future 模式配合使用的任务函数
 * 它负责执行函数并将结果设置到相关联的Future对象中。
 * \tparam Future Future模板类型
 * \tparam Function 要执行的函数类型
 */
template<
  template <typename> class Future,
  typename Function
>
class future_impl_task_func
{

public:
  /// 函数对象的结果类型（void，因为实际结果通过Future传递）
  typedef void result_type;
  /// 函数类型定义
  typedef Function function_type;
  /// Future结果类型定义（从函数返回类型推导）
  typedef typename result_of<function_type()>::type future_result_type;
  /// Future类型定义
  typedef Future<future_result_type> future_type;

  // 静态断言：任务函数必须是无参数函数
  BOOST_STATIC_ASSERT(function_traits<function_type()>::arity == 0);

  // 静态断言：任务函数的返回类型不能是void（需要有返回值传递给Future）
  BOOST_STATIC_ASSERT(!is_void<future_result_type>::value);

private:
  function_type             m_function;   // 要执行的函数对象
  shared_ptr<future_type>   m_future;     // 关联的Future对象智能指针

public:
  /* 构造函数
   * \param function 要执行的函数对象
   * \param future 关联的Future对象智能指针
   */
  future_impl_task_func(function_type const & function, shared_ptr<future_type> const & future)
  : m_function(function)
  , m_future(future)
  {
  }

  /* 函数调用操作符
   * 执行关联的函数对象并将结果设置到Future中。
   * 该方法会被线程池的工作线程调用来执行实际的任务。
   */
  void operator()()
  {
    // 检查函数对象是否有效
    if(m_function)
    {
      // 标记任务开始执行
      m_future->set_execution_status(true);
      
      // 检查任务是否已被取消
      if(!m_future->is_cancelled())
      {
        // TODO: 添加Future异常处理机制
        // 执行函数并将结果设置到Future中
        m_future->set_value(m_function());
      }
      
      // 标记任务执行结束（TODO: 考虑异常情况的处理）
      m_future->set_execution_status(false);
    }
  }

};
```

# 任务适配（`task_adaptors.hpp`）
实现了多种任务函数对象，包括普通任务、优先级任务和循环任务。

## 语法相关
### function0<void>
function0 是 **Boost.Function** 库中的一个模板类。
- 用来包装任何不接受任何参数的函数或函数对象

这里的 `<void>` 表示它所包装的函数或函数对象的返回类型是 void。
### thread::sleep 和 thread::yield 的区别
操作系统有一个核心组件叫调度器，调度器使用复杂的算法，如 *优先级调度*、*轮转调度*、*多级反馈队列*等，来决定下一个运行的线程。

当一个线程的时间片用完，调度器会进行上下文切换，保存当前线程的状态，并加载下一个要运行的线程的状态来运行。

`thread::sleep` 和 `thread::yield` 都是 **Boost.Thread** 库中用于控制线程执行的函数。
- `thread::sleep`
  - 当线程 A 调用 sleep(10ms) 后，它会进入 *阻塞* 状态，在这种状态下线程 A 会被从调度器的就绪队列中移除，不再参与 CPU 时间片的竞争。
  - 当 10ms 的休眠时间结束后，线程 A 会从 *阻塞* 状态转变为 *就绪* 状态，并被重新放回到调度器的就绪队列中。
- `thread::yield`
  - 线程 A 正在运行：线程 A 正在执行其代码，占用了 CPU。
  - 调用 yield：线程 A 主动调用 thread::yield()。
  - 进入就绪状态：线程 A 的状态从“运行”变为“就绪”，它被放回到调度器的就绪队列中
  - 调度器选择：操作系统调度器会立即从就绪队列中选择另一个线程（可能是线程 B、C 等，甚至有可能再次选择线程 A，虽然可能性很小）来运行

## 标准函数任务对象 `task_func`
```cpp
typedef function0<void> task_func;
```

## 优先级任务函数对象 `prio_task_func`
```cpp
/* 优先级任务函数对象
 * 封装了一个task_func对象并为其绑定了优先级 */ 
class prio_task_func
{
    private:
        unsigned int m_priority;  // 任务函数的优先级
        task_func m_function;     // 任务函数对象

    public:
        /// 表示函数对象的返回类型
        typedef void result_type;

    public:
        /* 构造函数，创建一个带优先级的任务函数对象 */
        prio_task_func(unsigned int const priority, task_func const & function)
            : m_priority(priority)
            , m_function(function)
        {
        }

        /*执行任务函数，调用封装的任务函数。如果函数对象有效，则执行它 */
        void operator() (void) const
        {
            if(m_function)
            {
                m_function();
            }
        }

        /* 比较操作符，基于优先级实现偏序关系
        * \param rhs 要比较的右操作数对象
        * \return 如果当前对象的优先级小于右操作数的优先级则返回true，否则返回false
        */
        bool operator< (const prio_task_func& rhs) const
        {
            return m_priority < rhs.m_priority; 
        }
};
```

## 循环任务函数对象 `looped_task_func`
```cpp
/* 循环任务函数对象，该函数对象封装了一个返回布尔值的线程函数对象。通过调用operator()来调用
* 被封装的任务函数，并且它会以固定的时间间隔执行，直到返回false为止。
* 间隔长度可以为零。请注意，只要任务在循环中，线程池的一个线程就会被占用 */ 
class looped_task_func
{
    private:
        function0<bool> m_function;   // 任务函数对象，返回 bool值控制是否继续循环
        unsigned int m_break_s;       // 间隔时间的秒数部分
        unsigned int m_break_ns;      // 间隔时间的纳秒数部分

    public:
        /// 表示函数对象的返回类型
        typedef void result_type;

    public:
        /* 构造函数，创建一个循环任务函数对象。
         \param function 要循环执行的任务函数对象，返回false时停止循环
         \param interval 最小间隔时间（毫秒），在第一次执行任务函数之前和后续执行之间的等待时间 */
        looped_task_func(function0<bool> const & function, unsigned int const interval = 0)
            : m_function(function)
        {
            // 将毫秒转换为秒和纳秒
            m_break_s  = interval / 1000;
            m_break_ns = (interval - m_break_s * 1000) * 1000 * 1000;
        }

        /* 执行循环任务函数
         * 执行封装的任务函数。首先等待指定的间隔时间，然后循环执行任务函数
         * 直到函数返回false。每次循环之间都会等待指定的间隔时间，如果间隔为0则调用thread::yield()让出CPU时间片。
         */
        void operator() (void) const
        {
            if(m_function)
            {
                // 第一次执行前先等待指定时间
                if(m_break_s > 0 || m_break_ns > 0)
                {
                    xtime xt;
                    xtime_get(&xt, boost::TIME_UTC_);
                    xt.nsec += m_break_ns;
                    xt.sec += m_break_s;
                    thread::sleep(xt); 
                }

                // 循环执行任务直到函数返回false
                while(m_function())
                {
                    if(m_break_s > 0 || m_break_ns > 0)
                    {
                        // 等待指定的间隔时间
                        xtime xt;
                        xtime_get(&xt, boost::TIME_UTC_);
                        xt.nsec += m_break_ns;
                        xt.sec += m_break_s;
                        thread::sleep(xt); 
                    }
                    else
                    {
                        // 如果没有间隔时间，让出CPU时间片给其他线程
                        thread::yield();
                    }
                }
            }
        }

}; // looped_task_func
```

# 线程池

## detail/pool_core.hpp

### 语法相关

#### 友元
`friend class` 是一种打破封装的机制：
- 类 A 中使用 `friend class` 声明类 B 后，类 B 中可以访问类 A 的所有 private 和 protected 成员
- 这种情况下，类 A 称为 ***授权类***，而类 B 称为 ***友元类***

#### 互斥锁封装器 lock_guard/unique_lock
lock_guard/unique_lock 都是 ***互斥锁封装器***，可以解决手动上锁/解锁的麻烦
- 使用一个锁来创建 *互斥锁封装器* 时会自动为该锁上锁，而析构时也会自动为对应的锁解锁
    ```cpp
    std::mutex cout_mutex;
    {
        std::lock_guard<std::mutex> lg(cout_mutex); // lg 创建完毕后，cout_mutex 上锁
    }
    // lock_guard 离开作用域即析构，cout_mutex 解锁
    ```
unique_lock 除此以外比 lock_guard 拥有更多功能
- 可以手动为对应的锁加锁 `lock()` 或解锁 `unlock()`
- 经常与 ***条件变量*** 配合使用
- 等其它

#### 条件变量
设计思想：让线程基于某些条件进行等待
- 一个条件变量和一个 unique_lock 互斥锁封装器配合使用，核心操作是 *等待* 和 *通知*
    ```cpp
    std::mutex mtx
    std::condition_variable cv
    std::unique_lock<std::mutex> lk(mtx)
    ```
- 某些线程 *等待* ：`cv.wait(lk, 可选：返回值为 bool 的函数 f)`，该操作包括
  - 进行 lk.unlock() 也就是对 mtx 解锁
  - 在 cv 上阻塞
- 一个线程 *通知* ：`cv.notify_one/all()`
  - `notify_one`：从所有正在该 cv 上等待的线程中，随机唤醒一个
  - `notify_all`：唤醒所有正在该 cv 上等待的线程
- 当某个在 cv 上阻塞的线程被通知后
  - lk.lock() 也就是对 mtx 加锁
  - 如果可选 f 返回 false
    - 重新 lk.unlock() 然后继续在 cv 上阻塞
  - 否则
    - 解除 cv 上的阻塞

#### 关键字 `mutable`
在类中，如果某成员函数被声明为 const，例如 `void f() const;`
- 相当于向编译器做出了一个承诺：该函数不会修改这个类的对象的任何成员变量

对于 const 函数，有时候为了实现线程安全，必须在函数内部对一个 mutex 成员变量进行加锁和解锁
- 这种情况下编译器会报错，因为在一个 const 函数里修改了成员变量。

`mutable` 关键字相当于 ***const 豁免权***。
- 用 mutable 来声明一个成员变量时，相当于告诉编译器：*这个成员变量，即使在 const 成员函数中，也可以被修改*

#### 递归互斥锁 `recursive_mutex`
考虑场景：一个线程要对同一个互斥锁多次加锁
- 使用普通互斥锁 `mutex` 时，后续加锁操作进行的前提是该锁已经解锁，于是卡住，造成 *死锁*
- `recursive_mutex` 是带计数的互斥锁：
  - 允许重复加锁（计数器加1），也就是加锁时不需要当前锁是解锁状态（计数器值为0）
  - 只有当计数器减到 0 时，这个锁才会被完全释放，其他线程才能获得这个锁

### 线程池类 `pool_core`

#### 成员
```cpp
private: // 以下成员可能被多个线程同时访问，使用volatile确保可见性
    volatile size_t m_worker_count;	        // 当前工作线程数量
    volatile size_t m_target_worker_count;	// 目标工作线程数量
    volatile size_t m_active_worker_count;  // 当前活动工作线程数量
private: // 以下成员一次只能被一个线程访问
    scheduler_type  m_scheduler;                                // 任务调度器实例
    scoped_ptr<size_policy_type> m_size_policy;                // 大小策略智能指针（永不为空）
    bool  m_terminate_all_workers;                              // 标示是否触发了所有工作线程的终止
    std::vector<shared_ptr<worker_type> > m_terminated_workers; // 已终止但未完全销毁的工作线程列表
private: // 以下成员实现了线程安全
    mutable recursive_mutex  m_monitor;                         // 递归互斥锁，保护共享资源
    mutable condition m_worker_idle_or_terminated_event;	    // 条件变量：工作线程空闲或已终止
    mutable condition m_task_or_terminate_workers_event;        // 条件变量：有任务可用或应减少工作线程总数
```

以上成员构成了 **线程池**
- `m_scheduler` 可以理解为 **任务队列**（存储需要处理的任务）
  - 包含 `std::deque<task_type> m_container`
- `m_size_policy`：**大小策略**（参考 [size_policies.hpp](./size_policies.hpp) 中的 `static_size`）智能指针
- `m_monitor` 是保护线程池的锁：任何可能修改线程池成员的操作，在进行前都要加锁
- `m_worker_count`：当前线程数
- `m_target_worker_count`：目标线程数
- `m_active_worker_count`：当前不在休息的线程数
- `m_worker_idle_or_terminated_event`：有线程空闲或终止
- `m_task_or_terminate_workers_event`：任务队列添加了新任务或终止所有线程

#### 构造函数
```cpp
pool_core()
    : m_worker_count(0)           // 初始化当前工作线程数为0
    , m_target_worker_count(0)    // 初始化目标工作线程数为0
    , m_active_worker_count(0)    // 初始化活动工作线程数为0
    , m_terminate_all_workers(false)  // 初始化终止标志为false
{
    // 获取自身的volatile引用用于创建大小策略
    pool_type volatile & self_ref = *this;
    m_size_policy.reset(new size_policy_type(self_ref));
    // 清空调度器，确保初始状态
    m_scheduler.clear();
}
```

#### 创建大小策略控制器并返回 `size_controller`
参考 [size_policies.hpp](./size_policies.hpp) 中的 `resize_controller` 和 `empty_controller`
```cpp
size_controller_type size_controller()
{
    return size_controller_type(*m_size_policy, this->shared_from_this());
}
```

#### 关闭线程池 `shutdown`
参考 [shutdown_policies.hpp](./shutdown_policies.hpp) 中的关闭策略 `wait_for_all_tasks`，`wait_for_active_tasks` 和 `immediately`
```cpp
void shutdown()
{
    ShutdownPolicy<pool_type>::shutdown(*this);
}
```

#### 向任务队列添加新任务并通知 `schedule`
```cpp
/* 向任务队列添加新任务并通知 */  
bool schedule(task_type const & task) volatile
{	
    // 使用锁定指针确保线程安全访问
    locking_ptr<pool_type, recursive_mutex> lockedThis(*this, m_monitor); 
    
    // 尝试将任务推入调度器
    if(lockedThis->m_scheduler.push(task))
    {
        // 通知一个工作线程有新任务可用
        lockedThis->m_task_or_terminate_workers_event.notify_one();
        return true;
    }
    else
    {
        return false;
    }
}
```

#### wait
```cpp
/* 阻塞当前执行线程，直到所有活动和待执行任务的总数等于或小于给定的阈值 */     
void wait(size_t const task_threshold = 0) const volatile
{
    const pool_type* self = const_cast<const pool_type*>(this);
    recursive_mutex::scoped_lock lock(self->m_monitor);

    // 如果阈值为0，等待所有任务完成
    if(0 == task_threshold)
    {
        while(0 != self->m_active_worker_count || !self->m_scheduler.empty())
        { 
            // 等待工作线程空闲或终止事件
            self->m_worker_idle_or_terminated_event.wait(lock);
        }
    }
    else
    {
        // 等待任务总数降到阈值以下
        while(task_threshold < self->m_active_worker_count + self->m_scheduler.size())
        { 
            // 等待工作线程空闲或终止事件
            self->m_worker_idle_or_terminated_event.wait(lock);
        }
    }
}

/* 阻塞当前执行线程，直到达到时间戳或所有活动和待执行任务的总数 
 * \param timestamp 函数最晚返回的时间戳
 * \param task_threshold 线程池和调度器中任务的最大数量，默认为0
 * \return 如果任务总数等于或小于阈值则返回true，否则返回false（超时）
 */     
bool wait(xtime const & timestamp, size_t const task_threshold = 0) const volatile
{
    const pool_type* self = const_cast<const pool_type*>(this);
    recursive_mutex::scoped_lock lock(self->m_monitor);

    // 如果阈值为0，等待所有任务完成
    if(0 == task_threshold)
    {
        while(0 != self->m_active_worker_count || !self->m_scheduler.empty())
        { 
            // 带超时的等待，如果超时则返回false
            if(!self->m_worker_idle_or_terminated_event.timed_wait(lock, timestamp)) return false;
        }
    }
    else
    {
        // 等待任务总数降到阈值以下
        while(task_threshold < self->m_active_worker_count + self->m_scheduler.size())
        { 
            // 带超时的等待，如果超时则返回false
            if(!self->m_worker_idle_or_terminated_event.timed_wait(lock, timestamp)) return false;
        }
    }

    // 成功等到条件满足
    return true;
}
```

#### 调整线程池中工作线程的数量 `resize`
流程：
- 如果已经终止所有工作线程：返回 false；否则将目标线程数设为传入的参数
- 如果当前线程数小于等于目标线程数
  - while(当前线程数小于目标线程数)
    - 使用类 `worker_thread` 的静态方法 `create_and_attach` 来基于 `boost::thread` 创建新线程，并异步执行 `run`
      - `worker_thread<pool_type>::create_and_attach(lockedThis->shared_from_this())`
        - 创建新的线程：`shared_ptr<worker_thread> worker(new worker_thread(pool))`
        - 对新创建的线程启动 run：`worker->m_thread.reset(new boost::thread(bind(&worker_thread::run, worker)))`
          - 注意这里启动的新线程是异步的！也就是 `create_and_attach` 会继续执行，而不会等待启动的这个线程执行完毕
            ```cpp
            void run()
            { 
                // 创建异常通知守卫，确保在异常情况下通知线程池
                scope_guard notify_exception(bind(&worker_thread::died_unexpectedly, this));
                // 持续执行任务直到线程池指示停止
                while(m_pool->execute_task()) {}
                // 正常退出时禁用异常通知守卫
                notify_exception.disable();
                // 通知线程池工作线程正常销毁
                m_pool->worker_destructed(this->shared_from_this());
            }
            ```
  - 当前线程数、当前不在休息线程数加 1
- 否则：条件变量 `m_task_or_terminate_workers_event`（有新任务或终止线程）通知所有
- 返回 `true`
    
```cpp
/* 调整线程池中工作线程的数量
 * \param worker_count 新的工作线程数量
 * \return 如果线程池将被调整大小则返回true，否则返回false
 */
bool resize(size_t const worker_count) volatile
{
    locking_ptr<pool_type, recursive_mutex> lockedThis(*this, m_monitor); 

    // 检查是否已经开始终止所有工作线程
    if(!m_terminate_all_workers)
    {
        // 设置新的目标工作线程数量
        m_target_worker_count = worker_count;
    }
    else
    { 
        // 如果正在终止，则不允许调整大小
        return false;
    }

    // 如果需要增加工作线程数量
    if(m_worker_count <= m_target_worker_count)
    { 
        // 创建新的工作线程直到达到目标数量
        while(m_worker_count < m_target_worker_count)
        {
            try
            {
                // 创建并附加新的工作线程
                worker_thread<pool_type>::create_and_attach(lockedThis->shared_from_this());
                m_worker_count++;       // 增加工作线程计数
                m_active_worker_count++;// 增加活动线程计数
            }
            catch(thread_resource_error)
            {
                // 如果线程资源不足，返回失败
                return false;
            }
        }
    }
    else
    { 
        // 如果需要减少工作线程数量，通知工作线程检查终止条件
        // TODO: 优化通知的工作线程数量
        lockedThis->m_task_or_terminate_workers_event.notify_all();
    }

    return true;
}
```

#### 从任务队列取任务执行 `execute_task`
流程：
- task 用于存放从线程池的任务队列中取出的任务
- { 即将访问线程池，创建线程池的锁 m_monitor 的封装器 lock（自动加锁）
- while(当前线程池的任务队列为空)
  - 如果当前工作线程的数量超出限制
    - 返回 false
  - 否则
    - 活动工作线程数减 1
    - 条件变量 `m_worker_idle_or_terminated_event`（当前有睡眠线程） 进行全部广播
    - 等待：直到被条件变量 `m_task_or_terminate_workers_event`（有新任务或终止线程）通知
    - 活动工作线程数加 1
- 取出线程池的任务队列的队首任务给 task }，该作用域结束，所以锁封装器 lock 销毁（m_monitor 自动解锁）
- 如果 task 存在则执行它
- 返回 true

```cpp
/* 执行单个任务，工作线程的主要执行函数。
* \return 如果成功执行任务则返回true，如果工作线程应该终止则返回false
*/
bool execute_task() volatile
{
    function0<void> task;

    {   // 获取任务的作用域
        pool_type* lockedThis = const_cast<pool_type*>(this);
        recursive_mutex::scoped_lock lock(lockedThis->m_monitor);

        // 如果需要减少线程数量，终止当前工作线程
        if(m_worker_count > m_target_worker_count)
        {	
            return false;	// 终止工作线程
        }

        // 等待任务可用
        while(lockedThis->m_scheduler.empty())
        {	
            // 再次检查是否需要减少工作线程数量
            if(m_worker_count > m_target_worker_count)
            {	
                return false;	// 终止工作线程
            }
            else
            {
                // 标记工作线程为空闲状态
                m_active_worker_count--;
                lockedThis->m_worker_idle_or_terminated_event.notify_all();
                
                // 等待新任务或终止信号
                lockedThis->m_task_or_terminate_workers_event.wait(lock);
                
                // 标记工作线程为活动状态
                m_active_worker_count++;
            }
        }

        // 从调度器中获取任务
        task = lockedThis->m_scheduler.top();
        lockedThis->m_scheduler.pop();
    }

    // 执行任务函数
    if(task)
    {
        task();
    }

    // 注释的守卫禁用代码，预留给将来的扩展
    //guard->disable();
    return true;
}
```

#### 终止所有工作线程 `terminate_all_workers`
流程：
- 设置 `m_terminate_all_workers`（终止所有工作线程标志位）为 true
- 设置目标线程数为 0
- 条件变量 `m_task_or_terminate_workers_event`（终止所有线程）进行全部广播
- 如果需要等待所有线程完全终止
  - while(当前活动线程数大于 0)
    - 等待：直到条件变量 `m_worker_idle_or_terminated_event`（有线程休眠或终止） 通知
  - 使用 for 等待 `m_terminated_workers`（被终止的线程）全部结束
  - 清空 `m_terminated_workers`
```cpp
/* 终止所有工作线程
 * \param wait 如果为true，则等待所有线程完全终止；否则立即返回
 */
void terminate_all_workers(bool const wait) volatile
{
    pool_type* self = const_cast<pool_type*>(this);
    recursive_mutex::scoped_lock lock(self->m_monitor);

    // 设置终止所有工作线程的标志
    self->m_terminate_all_workers = true;

    // 将目标工作线程数设为0
    m_target_worker_count = 0;
    // 通知所有工作线程检查终止条件
    self->m_task_or_terminate_workers_event.notify_all();

    // 如果需要等待线程完全终止
    if(wait)
    {
        // 等待所有活动工作线程终止
        while(m_active_worker_count > 0)
        {
            self->m_worker_idle_or_terminated_event.wait(lock);
        }

        // 等待所有已终止的工作线程完全销毁
        for(typename std::vector<shared_ptr<worker_type> >::iterator it = self->m_terminated_workers.begin();
            it != self->m_terminated_workers.end();
            ++it)
        {
            (*it)->join();  // 等待线程结束
        }
        // 清空已终止线程列表
        self->m_terminated_workers.clear();
    }
}
```

#### 处理线程意外死亡 `worker_died_unexpectedly`
```cpp
/* 处理工作线程意外死亡
 * 当工作线程因未处理的异常而意外死亡时调用此方法。更新线程计数并通知大小策略处理这种异常情况。
 * \param worker 意外死亡的工作线程智能指针
 */
void worker_died_unexpectedly(shared_ptr<worker_type> worker) volatile
{
    locking_ptr<pool_type, recursive_mutex> lockedThis(*this, m_monitor);

    // 减少工作线程计数
    m_worker_count--;
    m_active_worker_count--;
    // 通知等待的线程有工作线程死亡
    lockedThis->m_worker_idle_or_terminated_event.notify_all();	

    if(m_terminate_all_workers)
    {
        // 如果正在终止所有工作线程，将死亡的线程加入终止列表
        lockedThis->m_terminated_workers.push_back(worker);
    }
    else
    {
        // 否则通知大小策略处理工作线程意外死亡
        lockedThis->m_size_policy->worker_died_unexpectedly(m_worker_count);
    }
}
```

#### 处理线程正常销毁 `worker_destructed`
```cpp
/* 处理工作线程正常销毁
 * 当工作线程正常完成并准备销毁时调用此方法。
 * \param worker 要销毁的工作线程智能指针
 */
void worker_destructed(shared_ptr<worker_type> worker) volatile
{
  locking_ptr<pool_type, recursive_mutex> lockedThis(*this, m_monitor);
  
  // 减少工作线程计数
  m_worker_count--;
  m_active_worker_count--;
  // 通知等待的线程有工作线程终止
  lockedThis->m_worker_idle_or_terminated_event.notify_all();	

  if(m_terminate_all_workers)
  {
    // 如果正在终止所有工作线程，将销毁的线程加入终止列表
    lockedThis->m_terminated_workers.push_back(worker);
  }
}
```

## detail/worker_thread.hpp

### 语法相关

#### 作用域中的内存以及释放
在C++中，内存主要分为两块：栈 (Stack) 和 堆 (Heap)。
- `栈 (Stack)`：用于存储局部变量、函数参数等。**当一个函数或代码块结束时，它里面所有的局部变量都会被自动销毁**
    ```cpp
    int x = 10;
    MyClass obj;
    std::shared_ptr<A> a;  // 注意：a 这个智能指针变量本身在栈上
    ```
- `堆 (Heap)`：由程序员手动管理。可以随时在堆上申请内存，但**必须在用完后手动释放它，否则就会内存泄漏**。
  - 通过 `new` 或 `malloc` 申请的内存都在堆上。

#### shared_ptr
为什么需要 shared_ptr
- 传统的 C++ 编程中，管理动态分配的内存非常容易出错：需要记住何时调用 new，以及何时调用 delete
- 如果一个对象被多个指针引用，手动管理其生命周期会变得非常复杂：容易导致内存泄漏（忘记 delete）或二次释放（重复 delete）。

shared_ptr 是一种智能指针，允许多个指针安全地指向并共享同一个对象，当最后一个 shared_ptr 被销毁时，这个对象才会自动释放
- 核心机制为 *引用计数*：除了指向对象的原始指针外，每个 shared_ptr 实例都共享一个引用计数
- 当一个 shared_ptr 被创建、复制或作为函数参数传递时，引用计数会递增；当一个 shared_ptr 离开作用域或被重置时，引用计数会递减。
- 例子：
    ```cpp
    class MyClass {
    public:
        MyClass() {}
        ~MyClass() {}
    };

    int main() {
        std::shared_ptr<MyClass> p1; // 引用计数：0
        {
            std::shared_ptr<MyClass> p2(new MyClass()); // 引用计数：1
            p1 = p2; // p1 和 p2 共享所有权。引用计数：2
            std::shared_ptr<MyClass> p3 = p2; // p3 也共享所有权。引用计数：3
        } // p3 和 p2 离开作用域，引用计数递减到 1
    } // p1 离开作用域，引用计数递减到 0，对象被销毁
    ```
对于 `std::shared_ptr<A> a = std::make_shared<A>()`
- 创建的 A 对象实例是在堆上
- 而 a 是在栈上

#### scoped_ptr
`scoped_ptr` 是一种智能指针，相较于 `shared_ptr` 存在两个区别
- 独占所有权：任何时候，一个对象只能被一个 scoped_ptr 指向
- 不能对 scoped_ptr 进行拷贝构造或赋值操作

#### shared_ptr 的*循环引用*问题
例如：
```cpp
class B;

class A {
public:
    std::shared_ptr<B> ptr;
    A() {}
    ~A() {}
};

class B {
public:
    std::shared_ptr<A> ptr;
    B() {}
    ~B() {}
};

int main() {
    {
        // 注意 a 和 b 是在栈上，而创建的 A 和 B 的实例是在堆上
        std::shared_ptr<A> a = std::make_shared<A>(); // a 指向的 A 对象，引用计数为 1
        std::shared_ptr<B> b = std::make_shared<B>(); // b 指向的 B 对象，引用计数为 1

        // 建立循环引用
        a->ptr = b; // b 指向的 B 对象，引用计数变为 2 (b 和 a->ptr)
        b->ptr = a; // a 指向的 A 对象，引用计数变为 2 (a 和 b->ptr)
    }
    // 代码块结束后，栈上的 a 和 b 会自动释放，所以：
    // a 指向的 A 对象，引用计数变为 1 (b->ptr)
    // b 指向的 B 对象，引用计数变为 1 (a->ptr)
    // 但是 A 和 B 的实例是在堆上，程序不会自动操作

    // 程序将在这里结束，但A和B的析构函数永远不会被调用，也就是 A 和 B 的实例永远存在于堆上
    return 0;
}
```

#### weak_ptr

| 特性 | std::shared_ptr (共同所有人) | std::weak_ptr (访客) |
|------|-------------------------|---------------------|
| **核心目的** | 共享所有权，管理对象生命周期 | 监视/观察对象，不影响其生命周期 |
| **引用计数** | 增加强引用计数 (use_count) | 不增加强引用计数 |
| **控制生命周期？** | 是。只要它存在，对象就不会被销毁。 | 否。它的存在与否，对象该销毁还是会销毁。 |
| **直接访问** | 可以。直接使用 -> 和 *。 | 不可以。必须先调用 lock()。 |
| **如何访问** | `ptr->DoSomething();` | `if (auto locked_ptr = weak.lock()) { locked_ptr->DoSomething(); }` |
| **主要用途** | 作为通用的资源管理者 | 解决 shared_ptr 的循环引用问题、实现缓存等需要非侵入式观察的场景 |

例子：解决 shared_ptr 的循环应用
```cpp
class A {
public:
    std::weak_ptr<A> ptr; // 使用 weak_ptr
    A() {}
    ~A() {}
};

int main() {
    {
        std::shared_ptr<A> a1 = std::make_shared<A>(); // 引用计数为 1
        std::shared_ptr<A> a2 = std::make_shared<A>(); // 引用计数为 1

        a1->ptr = a2;
        a2->ptr = a1;
        // 引用计数没有增加，仍然为 1，因为 ptr 是 weak_ptr
    }
    // 代码块结束后，栈上的 a1 和 a2 会自动释放，所以：
    // a1 指向的对象，引用计数变为 0，释放掉
    // a2 指向的对象，引用计数变为 0，释放掉

    return 0;
}
```

#### `class A: public enable_shared_from_this<A>`
考虑如下场景：
- 某类型 `A` 的对象 `a` 正在运行，在它的某个成员函数内部，需要把自己以 `shared_ptr<A>` 的形式传递给另一个函数
- 使用类似于 `shared_ptr<A> sptr(this)` 的方式是不合适的，这是因为：
  - 当根据一个对象创建一个 shared_ptr 时，会创建一个 *控制块*
  - 假设在外部有一个 shared_ptr 管理 a，而又在成员函数内 `shared_ptr<A>(this)`，会存在两个独立的控制块
  - 同一个对象同时被两个独立的控制块管理，那么两个 shared_ptr 的生命周期都结束时，会对同一个对象删除两次！

假设 A 继承了模板类即 `class A: public enable_shared_from_this<A>`
- 当在外部用 shared_ptr 管理一个继承了 `enable_shared_from_this` 的类的实例时：
  - shared_ptr 的构造函数会在 a 的 `enable_shared_from_this` 部分存储一个指向控制块的 weak_ptr
- 由于是 public 继承，A 会获得成员函数 `shared_from_this()`
- 当在对象内部调用 `this->shared_from_this()` 时，会利用内部存储的那个 weak_ptr 来创建一个新的 shared_ptr
  - 因为这个新的 shared_ptr 是从已有的控制块信息创建的，所以共享同一个控制块，并正确地将引用计数加 1
  - 这样无论调用多少次 `shared_from_this()`，所有产生的 shared_ptr 都指向同一个控制块。

### 工作线程类 `worker_thread`

```cpp
/* 工作线程类
 * worker_thread代表一个执行线程，工作线程附属于线程池并处理该线程池的任务。
 * 该类是一个辅助类，不能直接构造或访问。工作线程通过静态方法创建并附加到线程池。
 * \tparam Pool 线程池的类型
 * \see pool_core
 */ 
template <typename Pool>
class worker_thread: public enable_shared_from_this<worker_thread<Pool>>, private noncopyable
{
public:
    /// 线程池类型定义
    typedef Pool pool_type;

private:
    shared_ptr<pool_type>      m_pool;     // 指向创建该工作线程的线程池的智能指针
    shared_ptr<boost::thread>  m_thread;   // 指向执行运行循环的线程的智能指针

    /* 私有构造函数
     * 构造一个新的工作线程。该构造函数是私有的，只能通过静态方法（create_and_attach）创建工作线程
     * \param pool 指向父线程池的智能指针
     * \see create_and_attach函数
     */
    worker_thread(shared_ptr<pool_type> const & pool)
    : m_pool(pool)
    {
        assert(pool);  // 确保线程池指针有效（不能为空）
    }

    /* 通知运行循环中发生异常
     * 将当前对象（工作线程）的共享指针传递给线程池 m_pool，通知其当前工作线程意外死亡
     */
    void died_unexpectedly()
    {
        m_pool->worker_died_unexpectedly(this->shared_from_this());
    }

public:
    /* 顺序执行线程池的任务
    *
    * 工作线程的主要执行函数。该函数会持续从线程池中获取并执行任务，
    * 直到线程池指示停止。使用作用域守卫确保在异常情况下也能正确
    * 通知线程池工作线程的状态变化。
    */
    void run()
    { 
        // 创建异常通知守卫，确保在异常情况下通知线程池
        scope_guard notify_exception(bind(&worker_thread::died_unexpectedly, this));

        // 持续执行任务直到线程池指示停止
        while(m_pool->execute_task()) {}

        // 正常退出时禁用异常通知守卫
        notify_exception.disable();
        // 通知线程池工作线程正常销毁
        m_pool->worker_destructed(this->shared_from_this());
    }

    /* 等待工作线程结束
     * 阻塞调用线程直到该工作线程完成执行。这是线程同步的标准方法，
     * 确保在销毁工作线程对象前所有任务都已完成。
     */
    void join()
    {
        m_thread->join();
    }

    /* 创建新的工作线程并将其附加到线程池
     * 静态工厂方法，用于创建新的工作线程实例并将其附加到指定的线程池。
     * 该方法创建工作线程对象和底层的 boost::thread，并启动线程执行。
     * \param pool 指向线程池的智能指针
     */
    static void create_and_attach(shared_ptr<pool_type> const & pool)
    {
        // 创建新的工作线程实例
        shared_ptr<worker_thread> worker(new worker_thread(pool));
        if(worker)
        {
            // 创建并启动底层的boost::thread
            worker->m_thread.reset(new boost::thread(bind(&worker_thread::run, worker)));
        }
    }

};
```

## scheduling_policies.hpp
实现了多种任务函数对象，包括FIFO（先进先出）、LIFO（后进先出）和基于优先级排序的调度策略。

### FIFO（先进先出）调度策略 `fifo_scheduler`
```cpp
/* 实现FIFO（先进先出）排序的调度策略
 * 该容器实现了FIFO调度策略。第一个添加到调度器的任务将是第一个被移除的任务。
 * 处理按相同的顺序依次进行。FIFO代表"先进先出"。
 */ 
template <typename Task = task_func>  
class fifo_scheduler
{
    public:
        /// 调度器的任务类型定义
        typedef Task task_type;

    protected:
        /// 内部任务容器，使用双端队列实现FIFO语义
        std::deque<task_type> m_container;

    public:
    /* 向调度器添加新任务，将任务添加到队列的末尾，遵循FIFO原则。
     * \param task 要添加的任务对象
     */
    bool push(task_type const & task)
    {
        m_container.push_back(task);  // 添加到队列末尾
        return true;
    }

    /* 移除下一个应该执行的任务 */
    void pop()
    {
        m_container.pop_front();  // 从队列前端移除
    }

    /* 获取下一个应该执行的任务 */
    task_type const & top() const
    {
        return m_container.front();  // 返回队列前端的任务
    }

    /* 获取调度器中当前的任务数量 */
    size_t size() const
    {
        return m_container.size();
    }

    /* 检查调度器是否为空 */
    bool empty() const
    {
        return m_container.empty();
    }

    /* 从调度器中移除所有任务 */  
    void clear()
    {   
        m_container.clear();
    } 
};
```

### LIFO（后进先出）调度策略 `lifo_scheduler`
```cpp
/* LIFO（后进先出）排序的调度策略
 * 该容器实现了LIFO调度策略。最后添加到调度器的任务将是第一个被移除的任务。
 * LIFO代表"后进先出"，类似于栈的行为。
 */ 
template <typename Task = task_func>  
class lifo_scheduler
{
    public:
        /// 调度器的任务类型定义
        typedef Task task_type;

    protected:
        /// 内部任务容器，使用双端队列实现LIFO语义
        std::deque<task_type> m_container;

    public:
    /* 向调度器添加新任务
     * 将任务添加到队列的前端，遵循LIFO原则。
     * \param task 要添加的任务对象
     */
    bool push(task_type const & task)
    {
        m_container.push_front(task);  // 添加到队列前端（栈顶）
        return true;
    }

    /* 移除下一个应该执行的任务 */
    void pop()
    {
        m_container.pop_front();  // 从队列前端移除（栈顶弹出）
    }

    /* 获取下一个应该执行的任务 */
    task_type const & top() const
    {
        return m_container.front();  // 返回队列前端的任务（栈顶）
    }

    /* 获取调度器中当前的任务数量 */
    size_t size() const
    {
        return m_container.size();
    }

    /* 检查调度器是否为空 */
    bool empty() const
    {
        return m_container.empty();
    }

    /* 从调度器中移除所有任务 */  
    void clear()
    {    
        m_container.clear();
    } 

};
```

### 基于优先级排序的调度策略 `prio_scheduler`
```cpp
/* 基于优先级排序的调度策略
 * 该容器实现了基于任务优先级的调度策略。具有最高优先级的任务将是第一个被移除的任务。
 * 任务对象必须支持使用operator<进行比较。operator<必须实现偏序关系。
 * \tparam Task 实现operator()和operator<的函数对象类型。operator<必须是偏序关系
 * 参考 prio_task_func
 */ 
template <typename Task = prio_task_func>  
class prio_scheduler
{
    public:
        /// 调度器的任务类型定义
        typedef Task task_type;

    protected:
        /// 内部任务容器，使用优先级队列实现基于优先级的调度
        std::priority_queue<task_type> m_container;

    public:
    /* 向调度器添加新任务
     * 将任务添加到优先级队列中，任务会根据其优先级自动排序。
     * \param task 要添加的任务对象
     */
    bool push(task_type const & task)
    {
        m_container.push(task);  // 添加到优先级队列，自动按优先级排序
        return true;
    }

    /* 移除下一个应该执行的任务 */
    void pop()
    {
        m_container.pop();  // 移除优先级最高的任务
    }

    /* 获取下一个应该执行的任务 */
    task_type const & top() const
    {
        return m_container.top();  // 返回优先级最高的任务
    }

    /* 获取调度器中当前的任务数量 */
    size_t size() const
    {
        return m_container.size();
    }

    /* 检查调度器是否为空 */
    bool empty() const
    {
        return m_container.empty();
    }

    /* 从调度器中移除所有任务 */  
    void clear()
    {    
        // 循环弹出所有任务直到队列为空
        while(!m_container.empty())
        {
        m_container.pop();
        }
    } 
};
```

## shutdown_policies.hpp
实现了线程池关闭策略，包括等待所有任务完成、等待活动任务完成和立即关闭的线程池关闭策略。

### 等待所有任务完成的线程池关闭策略 `wait_for_all_tasks`
```cpp
/* 等待所有任务完成的线程池关闭策略
 * 该关闭策略会等待所有任务（包括队列中的待执行任务和正在执行的任务）完成后，
 * 再终止所有工作线程。这是最安全的关闭策略，确保没有任务丢失。
 * \tparam Pool 线程池的核心类型
 */ 
  template<typename Pool>
  class wait_for_all_tasks
  {
  public:
    /* 执行关闭操作
     * 等待所有任务完成，然后终止所有工作线程。
     * \param pool 要关闭的线程池引用
     */
    static void shutdown(Pool& pool)
    {
      pool.wait();                      // 等待所有任务完成
      pool.terminate_all_workers(true); // 等待工作线程终止
    }
  };
```

### 等待活动任务完成的关闭策略的线程池关闭策略 `wait_for_active_tasks`
```cpp
/* 等待活动任务完成的关闭策略
 * 该关闭策略会清除队列中的待执行任务，只等待当前正在执行的任务完成，
 * 然后终止所有工作线程。这种策略会丢弃队列中未执行的任务。
 * \tparam Pool 线程池的核心类型
 */ 
  template<typename Pool>
  class wait_for_active_tasks
  {
  public:
    /* 执行关闭操作
     * 清除待执行任务队列，等待当前活动任务完成，然后终止所有工作线程。
     * \param pool 要关闭的线程池引用
     */
    static void shutdown(Pool& pool)
    {
      pool.clear();                     // 清除队列中的待执行任务
      pool.wait();                      // 等待当前活动任务完成
      pool.terminate_all_workers(true); // 等待工作线程终止
    }
  };
```

### 立即关闭的线程池关闭策略 `immediately`
```cpp
/* 立即关闭策略
 * 不等待任何任务或工作线程终止，立即清除任务队列并终止线程池。
 * \tparam Pool 线程池的核心类型
 */ 
  template<typename Pool>
  class immediately
  {
  public:
    /* 执行关闭操作
     * 立即清除任务队列并终止所有工作线程，不等待任务完成。
     * \param pool 要关闭的线程池引用
     */
    static void shutdown(Pool& pool)
    {
      pool.clear();                      // 清除队列中的待执行任务
      pool.terminate_all_workers(false); // 不等待，立即终止工作线程
    }
  };
```

## size_policies.hpp
实现了一个大小策略类 `static_size`
- 持有一个对线程池的引用

以及线程大小控制器 `resize_controller` 和 `empty_controller`
- 不仅持有一个对线程池的引用，还持有一个对大小策略的引用

### 语法相关
#### typename Pool::size_policy_type
关于 `typename Pool::size_policy_type` 的解释
- Pool 是 `empty_controller` 类模板的一个模板参数，代表了一个具体的类型
- Pool::size_policy_type 表示要访问 Pool 类内部 size_policy_type（size_policy_type 是 Pool 类内部定义的 *类型别名（typedef）* 或 *静态成员变量*）
  - 关键在于，编译器在解析 template 模板时，并不知道 Pool 究竟是什么类型，所以无法确定 Pool::size_policy_type 是一个 *类型别名（typedef）*，还是一个 *静态成员变量*
  - 而 typename 的作用就是告诉编译器：Pool::size_policy_type 是一个 *类型别名（typedef）*
#### `std::reference_wrapper`
`std::reference_wrapper` 是一种 C++ 标准库中的类模板，主要作用是解决 C++ 引用不能被复制、不能被赋值，也不能存放在标准容器中的问题。

在 C++ 中，普通的引用（&）有几个严格的限制：
- 无法重新绑定： 引用一旦初始化，就不能再引用另一个对象。
- 不可复制： 引用本身不是一个对象，不能被复制构造或赋值。
- 不能存放在容器中： 由于不能被复制，你不能在像 std::vector 或 std::map 这样的标准容器中直接存放引用。

std::reference_wrapper 通过封装一个指针来模拟引用的行为，但它本身是一个可复制的对象。

通常使用两个辅助函数来创建 reference_wrapper：
- `std::ref(obj)`：创建一个 `std::reference_wrapper<T>` 对象，包装 obj。
- `std::cref(obj)`：创建一个 `std::reference_wrapper<const T>` 对象，包装 obj 的常量引用。
#### `reference_wrapper<Pool volatile>` 中的 `Pool volatile`
`volatile` 关键字
- 一种类型限定符，它告诉编译器，被该关键字修饰的变量的值可能会在任何时候被外部因素改变，而不仅仅是通过代码中的赋值语句。这些外部因素可能包括：
  - 多线程环境： 另一个线程修改了该变量。
  - 硬件操作： 内存映射的 I/O 端口，其值可能被硬件设备改变。
  - 中断服务程序： 一个异步中断改变了该变量的值。
- 当编译器看到 volatile 关键字时，它会关闭对该变量的一些优化措施：以确保每次访问该变量时都从内存中重新读取它的值，而不是使用寄存器中缓存的旧值。

在 `reference_wrapper<Pool volatile>` 中，`Pool volatile` 表示 reference_wrapper 所引用的对象类型是 `volatile Pool`。这意味着：
- 通过 reference_wrapper 访问 Pool 对象时，编译器会确保每次都从内存中读取该对象，避免使用缓存。
- 这通常发生在多线程编程中，Pool 对象可能被多个线程同时访问和修改，其中一个线程可能正在使用 reference_wrapper 来操作它

### 空的线程池大小控制器 `empty_controller`
```cpp
/* 空的大小控制器，不提供任何功能，用于不需要动态调整线程池大小的场景 */ 
template<typename Pool>
struct empty_controller
{
  /* 构造函数
   * 创建一个空的控制器，不执行任何操作。
   * \param policy 大小策略引用（未使用）
   * \param pool 线程池智能指针（未使用）*/
  empty_controller(typename Pool::size_policy_type&, shared_ptr<Pool>) {}
};
```

### 支持调整大小的线程池大小控制器 `resize_controller`
```cpp
/* 支持调整大小的策略控制器
 * 该控制器允许在运行时动态调整线程池的大小。它持有对大小策略的引用和对线程池的智能指针，确保在控制器存在期间线程池保持有效 */ 
template< typename Pool >
class resize_controller
{
  typedef typename Pool::size_policy_type size_policy_type;   // 大小策略类型定义
  reference_wrapper<size_policy_type> m_policy;              // 大小策略的引用包装器
  shared_ptr<Pool> m_pool;                                   // 线程池智能指针，确保线程池在控制器存在期间保持有效

public:
  /* 构造函数，创建一个可调整大小的控制器。
    * \param policy 大小策略引用
    * \param pool 线程池智能指针 */
  resize_controller(size_policy_type& policy, shared_ptr<Pool> pool)
    : m_policy(policy)
    , m_pool(pool)
  {
  }

  /* 调整线程池大小，通过大小策略调整线程池中工作线程的数量。
    * \param worker_count 新的工作线程数量
    * \return true表示调整成功，false表示调整失败 */
  bool resize(size_t worker_count)
  {
    return m_policy.get().resize(worker_count);
  }
};
```

### 大小策略类 `static_size`
```cpp
/* 固定大小策略，但支持手动调整和异常恢复 */ 
template<typename Pool>
class static_size
{
  reference_wrapper<Pool volatile> m_pool;  ///< 线程池的引用包装器（volatile确保线程安全）

public:
  /* 初始化线程池大小
   * 静态方法，用于初始化线程池的工作线程数量。
   * \param pool 线程池引用
   * \param worker_count 初始工作线程数量 */
  static void init(Pool& pool, size_t const worker_count)
  {
    pool.resize(worker_count);  // 调整线程池大小到指定数量
  }

  /* 构造函数，创建一个静态大小策略对象。
   * \param pool 线程池的volatile引用 */
  static_size(Pool volatile & pool)
    : m_pool(pool)
  {}

  /* 调整线程池大小，手动调整线程池中工作线程的数量。
   * \param worker_count 新的工作线程数量
   * \return true表示调整成功，false表示调整失败 */
  bool resize(size_t const worker_count)
  {
    return m_pool.get().resize(worker_count);
  }

  /* 处理工作线程意外退出
   * 当工作线程意外退出时调用此方法。策略会自动创建一个新线程来替换退出的线程，以维持线程池的稳定性。
   * \param new_worker_count 当前的工作线程数量 */
  void worker_died_unexpectedly(size_t const new_worker_count)
  {
    m_pool.get().resize(new_worker_count + 1);  // 创建新线程替换退出的线程
  }

  /* 任务调度通知（待实现）
   * 当有任务被调度时调用。目前为空实现，预留给将来的扩展 */
  void task_scheduled() {}

  /* 任务完成通知（待实现）
   * 当任务完成时调用。目前为空实现，预留给将来的扩展 */
  void task_finished() {}
};
```

## pool.hpp

### 相关语法

#### `m_shutdown_controller(static_cast<void*>(0), bind(&pool_core_type::shutdown, m_core)`
智能指针 `shared_ptr` 的构造函数有一个重载版本 `shared_ptr<T> p(pointer, deleter)`
- pointer：是 shared_ptr 要“管理”的指针。
- deleter：是一个可调用对象（比如函数、函数对象或 bind 的结果）
- *当引用计数变为 0 时，shared_ptr 不会调用 delete pointer，而是会调用 deleter(pointer)*

考虑构造函数
```cpp
thread_pool(size_t initial_threads = 0)
: m_core(new pool_core_type)
, m_shutdown_controller(static_cast<void*>(0), bind(&pool_core_type::shutdown, m_core))
{
size_policy_type::init(*m_core, initial_threads);
}
```
- `thread_pool` 被设计为可以自由复制的，例如
    ```cpp
    thread_pool pool_a(4);
    thread_pool pool_b = pool_a; // pool_b 和 pool_a 指向同一个线程池核心
    ```
- 当复制 thread_pool 对象时，内部的 m_core 和 m_shutdown_controller 这两个 shared_ptr 都会被复制，导致它们指向的对象的引用计数增加
- 只要还有一个 thread_pool 对象存在，m_core 和 m_shutdown_controller 的引用计数就大于 0
- 当最后一个 thread_pool 对象被销毁时，它内部的 m_core 和 m_shutdown_controller 也会被销毁
  - 于是会调用 `m_core->shutdown()`，确保线程池的关闭

### 线程池类 `thread_pool`
是 [detail/pool_core.hpp](./detail/pool_core.hpp) 中 `pool_core` 中的接口类。
包含成员：
- `shared_ptr<pool_core_type> m_core`：指向具体线程池的智能指针
- `shared_ptr<void> m_shutdown_controller`

```cpp
/* 线程池类
* 线程池是在同一进程内进行异步和并行处理的机制。thread_pool 类提供了将异步任务作为函数对象分发的便捷方式
*
* \tparam Task 实现operator 'void operator() (void) const'的函数对象。
*              线程池调用operator()来执行任务。异常将被忽略。
* \tparam SchedulingPolicy 决定任务如何调度的任务容器。保证此容器一次只能
*                          被一个线程访问。调度器不应抛出异常。
* \tparam SizePolicy 控制线程池大小的策略类
* \tparam SizePolicyController 大小策略控制器类
* \tparam ShutdownPolicy 线程池关闭策略类
*/ 
template <
  typename Task                                   = task_func,           // 任务类型，默认为task_func
  template <typename> class SchedulingPolicy      = fifo_scheduler,      // 调度策略，默认为FIFO
  template <typename> class SizePolicy            = static_size,         // 大小策略，默认为静态大小
  template <typename> class SizePolicyController  = resize_controller,   // 大小控制器，默认为可调整大小
  template <typename> class ShutdownPolicy        = wait_for_all_tasks   // 关闭策略，默认等待所有任务完成
> 
class thread_pool 
{
  typedef detail::pool_core<Task, 
                            SchedulingPolicy,
                            SizePolicy,
                            SizePolicyController,
                            ShutdownPolicy> pool_core_type;
  // 指向核心实现的智能指针，使用Pimpl惯用法
  shared_ptr<pool_core_type>          m_core;
  // 关闭控制器，当持有核心指针的最后一个线程池被删除时，控制器会关闭线程池
  shared_ptr<void>                    m_shutdown_controller;

public:
  typedef Task task_type;
  typedef SchedulingPolicy<task_type> scheduler_type;
  typedef SizePolicy<pool_core_type> size_policy_type; 
  typedef SizePolicyController<pool_core_type> size_controller_type;

public:
  /* 构造函数
   * 创建线程池并立即调整到指定的线程数量。线程池的实际线程数量取决于大小策略。
   * \param initial_threads 初始线程数量，默认为 0 
   */
  thread_pool(size_t initial_threads = 0)
  : m_core(new pool_core_type)
  , m_shutdown_controller(static_cast<void*>(0), bind(&pool_core_type::shutdown, m_core))
  {
    // 使用大小策略初始化线程池
    size_policy_type::init(*m_core, initial_threads);
  }

  /* 获取管理线程池中线程数量的大小控制器 */
  size_controller_type size_controller()
  {
    return m_core->size_controller();
  }

  /* 获取线程池中的线程数量 */
  size_t size() const
  {
    return m_core->size();
  }

  /* 向线程池 m_core 的任务队列中添加新任务 task 并通知所有工作线程 */  
  bool schedule(task_type const & task)
  {	
    return m_core->schedule(task);
  }

  /* 获取线程池中当前活动（正在执行）的任务数量 */  
  size_t active() const
  {
    return m_core->active();
  }

  /* 返回准备执行的任务数量 */  
  size_t pending() const
  {
    return m_core->pending();
  }

  /* 从线程池的调度器中移除所有待执行任务 */  
  void clear()
  { 
    m_core->clear();
  }    

  /* 检查线程池是否没有待执行的任务 */   
  bool empty() const
  {
    return m_core->empty();
  }	

  /* 阻塞当前执行线程，直到所有活动和待执行任务的总数等于或小于给定的阈值 */     
  void wait(size_t task_threshold = 0) const
  {
    m_core->wait(task_threshold);
  }	

  /* 阻塞当前执行线程，直到达到时间戳或所有活动和待执行任务的总数 */       
  bool wait(xtime const & timestamp, size_t task_threshold = 0) const
  {
    return m_core->wait(timestamp, task_threshold);
  }
};
```

# schedule 适配（pool_adaptors.hpp）
用户： 
- 调用 `schedule(pool, obj/task)`(来自 pool_adaptors.hpp/future.hpp) 
  - future.hpp 中的 schedule 使用了 Future 机制：用于某些任务需要返回结果
  - pool_adaptors.hpp 中的 schedule：适用于返回 void 的任务
- ——> 内部调用 `pool.schedule(task)` (来自 pool.hpp 的 thread_pool 的成员函数)
- ——> 委托给 `m_core->schedule(task)` (来自 detail/pool_core.hpp 的类 pool_core 的成员函数)
- ——> 向线程池的任务队列添加新任务并通知所有工作线程

## 向线程池 Pool 的任务队列添加新任务 bind(&Runnable::run, obj) 并通知
```cpp
/* 向线程池 Pool 的任务队列添加新任务 bind(&Runnable::run, obj) 并通知
* \tparam Pool 线程池类型
* \tparam Runnable 可运行对象类型，必须有run()成员函数
* \param pool 线程池引用
* \param obj Runnable对象的智能指针，其run()方法将被执行且不应抛出异常
* \return true表示任务调度成功，false表示调度失败
*/  
template<typename Pool, typename Runnable>
bool schedule(Pool& pool, shared_ptr<Runnable> const & obj)
{
    return pool->schedule(bind(&Runnable::run, obj));
}
```

## 向线程池 Pool 的任务队列添加新任务 task（返回类型void） 并通知
```cpp
/* 向线程池 Pool 的任务队列添加新任务 task（返回类型 void） 并通知
* 使用SFINAE技术确保只有返回void的任务函数才会匹配此重载。
* \tparam Pool 线程池类型
* \param pool 线程池引用
* \param task 任务函数对象
* \return true表示任务调度成功，false表示调度失败
*/  
template<typename Pool>
typename enable_if < 
    is_void< typename result_of< typename Pool::task_type() >::type >,
    bool
>::type
schedule(Pool& pool, typename Pool::task_type const & task)
{	
    return pool.schedule(task);
}

template<typename Pool>
typename enable_if < 
    is_void< typename result_of< typename Pool::task_type() >::type >,
    bool
>::type
schedule(shared_ptr<Pool> const pool, typename Pool::task_type const & task)
{	
    return pool->schedule(task);
}
```